# The author-topic model: LDA with metadata

**TODO:** StackExchange example probably won't be included, as I'm not having good results with it and cannot make the tag prediction to work at all. If it is not included, fix this section, and do similarity queries with authors.

In this tutorial, you will learn how to use the author-topic model in Gensim. First, we will apply it to a corpus consisting of scientific papers, to get insight about the authors of the papers. After that, we will apply the model on StackExchange posts with tags, and implement a simple automatic tagging system.

The author-topic model is in extension of Latent Dirichlet Allocation (LDA). Each document is associated with a set of authors, and the topic distributions for each of these authors are learned. Each author is also associated with multiple documents. To learn about the theoretical side of the author-topic model, see [Rosen-Zvi and co-authors](https://mimno.infosci.cornell.edu/info6150/readings/398.pdf), for example.

Naturally, familiarity with topic modelling, LDA and Gensim is assumed in this tutorial. If you are not familiar with either LDA, or its Gensim implementation, consider some of these resources:
* Gentle introduction to the LDA model: http://blog.echen.me/2011/08/22/introduction-to-latent-dirichlet-allocation/
* Gensim's LDA API documentation: https://radimrehurek.com/gensim/models/ldamodel.html
* Topic modelling in Gensim: http://radimrehurek.com/topic_modeling_tutorial/2%20-%20Topic%20Modeling.html
* Pre-processing and training LDA: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/lda_training_tips.ipynb

> NOTE:
>
> To run this tutorial on your own, install Gensim, SpaCy, Scikit-Learn, and Bokeh.

**FIXME:** technical details can be found at TODO 

In part 1 of this tutorial, we will illustrate basic usage of the model, and explore the resulting representation. How to load and pre-process the dataset used is also covered.

In part 2, we will develop a simple automatic tagging system, and some more of the model's functionality will be shown.

## Part 1: analyzing scientific papers

The data used in part 1 consists of scientific papers about machine learning, from the Neural Information Processing Systems conference (NIPS). It is the same dataset used in the [Pre-processing and training LDA](https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/lda_training_tips.ipynb) tutorial, mentioned earlier.

As in the LDA tutorial, we will be performing qualitative analysis of the model, and at times this will require an understanding of the subject matter of the data. If you try running this tutorial on your own, consider applying it on a dataset with subject matter that you are familiar with. For example, try one of the [StackExchange datadump datasets](https://archive.org/details/stackexchange).

You can download the data from Sam Roweis' website (http://www.cs.nyu.edu/~roweis/data.html).

In the following sections we will load the data, pre-process it, train the model, and explore the results using some of the implementation's functionality. Feel free to skip the loading and pre-processing for now, if you are familiar with the process.

### Loading the data

In the cell below, we crawl the folders and files in the dataset, and read the files into memory.

In [1]:
import os, re

# Folder containing all NIPS papers.
data_dir = '../../../../data/nipstxt/'  # Set this path to the data on your machine.

# Folders containin individual NIPS papers.
yrs = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
dirs = ['nips' + yr for yr in yrs]

# Get all document texts and their corresponding IDs.
docs = []
doc_ids = []
for yr_dir in dirs:
    files = os.listdir(data_dir + yr_dir)  # List of filenames.
    for filen in files:
        # Get document ID.
        (idx1, idx2) = re.search('[0-9]+', filen).span()  # Matches the indexes of the start end end of the ID.
        doc_ids.append(yr_dir[4:] + '_' + str(int(filen[idx1:idx2])))
        
        # Read document text.
        # Note: ignoring characters that cause encoding errors.
        with open(data_dir + yr_dir + '/' + filen, errors='ignore', encoding='utf-8') as fid:
            txt = fid.read()
            
        # Replace any whitespace (newline, tabs, etc.) by a single space.
        txt = re.sub('\s', ' ', txt)
        
        docs.append(txt)

Construct a mapping from author names to document IDs.

In [2]:
filenames = [data_dir + 'idx/a' + yr + '.txt' for yr in yrs]  # Using the years defined in previous cell.

# Get all author names and their corresponding document IDs.
author2doc = dict()
i = 0
for yr in yrs:
    # The files "a00.txt" and so on contain the author-document mappings.
    filename = data_dir + 'idx/a' + yr + '.txt'
    for line in open(filename, errors='ignore', encoding='utf-8'):
        # Each line corresponds to one author.
        contents = re.split(',', line)
        author_name = (contents[1] + contents[0]).strip()
        # Remove any whitespace to reduce redundant author names.
        author_name = re.sub('\s', '', author_name)
        # Get document IDs for author.
        ids = [c.strip() for c in contents[2:]]
        if not author2doc.get(author_name):
            # This is a new author.
            author2doc[author_name] = []
            i += 1
        
        # Add document IDs to author.
        author2doc[author_name].extend([yr + '_' + id for id in ids])

# Use an integer ID in author2doc, instead of the IDs provided in the NIPS dataset.
# Mapping from ID of document in NIPS datast, to an integer ID.
doc_id_dict = dict(zip(doc_ids, range(len(doc_ids))))
# Replace NIPS IDs by integer IDs.
for a, a_doc_ids in author2doc.items():
    for i, doc_id in enumerate(a_doc_ids):
        author2doc[a][i] = doc_id_dict[doc_id]

### Pre-processing text

The text will be pre-processed using the following steps:
* Tokenize text.
* Replace all whitespace by single spaces.
* Remove all punctuation and numbers.
* Remove stopwords.
* Lemmatize words.
* Add multi-word named entities.
* Add frequent bigrams.
* Remove frequent and rare words.

Part 2 will use the same pre-processing, for the most part, so we shall explain it here.

A lot of the heavy lifting will be done by the great package, Spacy. Spacy markets itself as "industrial-strength natural language processing", is fast, enables multiprocessing, and is easy to use. First, let's import it and load the NLP pipline in english.

In [3]:
import spacy
nlp = spacy.load('en')

In the code below, Spacy takes care of tokenization, removing non-alphabetic characters, removal of stopwords, lemmatization and named entity recognition.

Note that we only keep named entities that consist of more than one word, as single word named entities are already there.

**TODO:** use custom pipeline. Using the entire SpaCy pipeline is very expensive; for example, I'm probably doing POS tagging below, but not using it for anything.

In [4]:
%%time
processed_docs = []    
for doc in nlp.pipe(docs, n_threads=4, batch_size=100):
    # Process document using Spacy NLP pipeline.
    
    ents = doc.ents  # Named entities.

    # Keep only words (no numbers, no punctuation).
    # Lemmatize tokens, remove punctuation and remove stopwords.
    doc = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

    # Remove common words from a stopword list.
    #doc = [token for token in doc if token not in STOPWORDS]

    # Add named entities, but only if they are a compound of more than word.
    doc.extend([str(entity) for entity in ents if len(entity) > 1])
    
    processed_docs.append(doc)

CPU times: user 9min 8s, sys: 464 ms, total: 9min 9s
Wall time: 3min 3s


In [5]:
docs = processed_docs
del processed_docs

Below, we use a Gensim model to add bigrams. Note that this achieves the same goal as named entity recognition, that is, finding adjacent words that have some particular significance.

In [6]:
# Compute bigrams.
from gensim.models import Phrases
# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

/home/olavur/Dropbox/my_folder/workstuff/DTU/thesis/code/gensim/gensim/models/phrases.py:248: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


Now we are ready to construct a dictionary, as our vocabulary is finalized. We then remove common words (occurring $> 50\%$ of the time), and rare words (occur $< 20$ times in total).

In [7]:
# Create a dictionary representation of the documents, and filter out frequent and rare words.

from gensim.corpora import Dictionary
dictionary = Dictionary(docs)

# Remove rare and common tokens.
# Filter out words that occur too frequently or too rarely.
max_freq = 0.5
min_wordcount = 20
dictionary.filter_extremes(no_below=min_wordcount, no_above=max_freq)

_ = dictionary[0]  # This sort of "initializes" dictionary.id2token.

We produce the vectorized representation of the documents, to supply the author-topic model with, by computing the bag-of-words.

In [8]:
# Vectorize data.

# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

Let's inspect the dimensionality of our data.

In [9]:
print('Number of authors: %d' % len(author2doc))
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of authors: 2479
Number of unique tokens: 6996
Number of documents: 1740


### Train and use model

We train the author-topic model on the data prepared in the previous sections. 

The interface to the author-topic model is very similar to that of LDA in Gensim. In addition to a corpus, ID to word mapping (`id2word`) and number of topics (`num_topics`), the author-topic model requires either an author to document ID mapping (`author2doc`), or the reverse (`doc2author`).

Below, we have also (this can be skipped for now):
* Increased the number of `passes` over the dataset (to improve the convergence of the optimization problem).
* Decreased the number of `iterations` over each document (related to the above).
* Specified the mini-batch size (`chunksize`) (primarily to speed up training).
* Turned off bound evaluation (`eval_every`) (as it takes a long time to compute).
* Turned on automatic learning of the `alpha` and `eta` priors (to improve the convergence of the optimization problem).
* Set the random state (`random_state`) of the random number generator (to make these experiments reproducible).

We load the model, and train it.

In [10]:
from gensim.models import AuthorTopicModel
%time model = AuthorTopicModel(corpus=corpus, num_topics=10, id2word=dictionary.id2token, \
                author2doc=author2doc, chunksize=2000, passes=1, eval_every=0, \
                iterations=1, random_state=1)

CPU times: user 3.75 s, sys: 224 ms, total: 3.98 s
Wall time: 3.78 s


If you believe your model hasn't converged, you can continue training using `model.update()`. If you have additional documents and/or authors call `model.update(corpus, author2doc)`.

Before we explore the model, let's try to improve upon it. To do this, we will train several models with different random initializations, by giving different seeds for the random number generator (`random_state`). We evaluate the topic coherence of the model using the [top_topics](https://radimrehurek.com/gensim/models/ldamodel.html#gensim.models.ldamodel.LdaModel.top_topics) method, and pick the model with the highest topic coherence.



In [12]:
%%time
model_list = []
for i in range(5):
    model = AuthorTopicModel(corpus=corpus, num_topics=10, id2word=dictionary.id2token, \
                    author2doc=author2doc, chunksize=2000, passes=100, gamma_threshold=1e-10, \
                    eval_every=0, iterations=1, random_state=i)
    top_topics = model.top_topics(corpus)
    tc = sum([t[1] for t in top_topics])
    model_list.append((model, tc))

CPU times: user 11min 57s, sys: 2min 16s, total: 14min 13s
Wall time: 11min 38s


Choose the model with the highest topic coherence.

In [15]:
model, tc = max(model_list, key=lambda x: x[1])
print('Topic coherence: %.3e' %tc)

Topic coherence: -1.803e+03


We save the model, to avoid having to train it again, and also show how to load it again.

In [16]:
# Save model.
model.save('/tmp/model.atmodel')

In [17]:
# Load model.
model = AuthorTopicModel.load('/tmp/model.atmodel')

### Explore author-topic representation

Now that we have trained a model, we can start exploring the authors and the topics. First, let's simply print the most important words in the topics.

In [18]:
model.show_topics(num_topics=10)

[(0,
  '0.029*"image" + 0.006*"constraint" + 0.006*"component" + 0.006*"region" + 0.006*"matrix" + 0.006*"pixel" + 0.006*"solution" + 0.005*"surface" + 0.005*"object" + 0.005*"source"'),
 (1,
  '0.008*"optimal" + 0.007*"action" + 0.007*"q" + 0.007*"approximation" + 0.006*"policy" + 0.005*"gaussian" + 0.005*"w" + 0.005*"noise" + 0.005*"generalization" + 0.004*"convergence"'),
 (2,
  '0.014*"visual" + 0.012*"field" + 0.010*"object" + 0.009*"image" + 0.008*"map" + 0.008*"layer" + 0.008*"eye" + 0.008*"direction" + 0.008*"activity" + 0.008*"cell"'),
 (3,
  '0.022*"control" + 0.010*"dynamic" + 0.009*"analog" + 0.008*"circuit" + 0.008*"trajectory" + 0.007*"neuron" + 0.007*"chip" + 0.007*"controller" + 0.006*"motor" + 0.005*"gate"'),
 (4,
  '0.013*"neuron" + 0.009*"f" + 0.008*"p" + 0.008*"memory" + 0.007*"let" + 0.007*"w" + 0.007*"bound" + 0.007*"capacity" + 0.006*"theorem" + 0.006*"threshold"'),
 (5,
  '0.021*"neuron" + 0.021*"cell" + 0.010*"spike" + 0.010*"response" + 0.010*"signal" + 0.008*

These topics are by no means perfect. They have problems such as *chained topics*, *intruded words*, *random topics*, and *unbalanced topics* (see [Mimno and co-authors 2011](https://people.cs.umass.edu/~wallach/publications/mimno11optimizing.pdf)). They will do for the purposes of this tutorial, however.

**FIXME:** re-write the interpretation of the topics below, if necessary.

Below, we use the `model[name]` syntax to retrieve the topic distribution for some authors. Comparing the authors' topics with the topics above, we observe that the model has correctly identified that Yann LeCun and Geoffrey E. Hinton both have something to do with neural networks (topic 5), speech recognition (topic 1 and 5) and statistical machine learning (topic 9). We also observe that Yann LeCun has been particularly occupied with image processing, and perhaps that Geoffrey E. Hinton has worked with visual perception in neuroscience (this is less clear).

Similarly, Terrence J. Sejnowski and James M. Bower are both neuroscientist, first and foremost, and their topic distributions seem to reflect that.

**TODO:** try to make it less cumbersome to compare author-topics below and topics above.

In [19]:
from pprint import pprint

name = 'YannLeCun'
print('\n%s' % name)
print('Docs:', author2doc[name])
print('Topics:')
pprint(model[name])

name = 'GeoffreyE.Hinton'
print('\n%s' % name)
print('Docs:', author2doc[name])
print('Topics:')
pprint(model[name])

name = 'TerrenceJ.Sejnowski'
print('\n%s' % name)
print('Docs:', author2doc[name])
print('Topics:')
pprint(model[name])

name = 'JamesM.Bower'
print('\n%s' % name)
print('Docs:', author2doc[name])
print('Topics:')
pprint(model[name])


YannLeCun
Docs: [143, 406, 370, 495, 456, 449, 595, 616, 760, 752, 1532]
Topics:
[(8, 0.99976156441010589)]

GeoffreyE.Hinton
Docs: [56, 143, 284, 230, 197, 462, 463, 430, 688, 784, 826, 848, 869, 1387, 1684, 1728]
Topics:
[(1, 0.16022855953677534),
 (2, 0.18085978419244769),
 (8, 0.18989122578835921),
 (9, 0.46894303639673746)]

TerrenceJ.Sejnowski
Docs: [513, 530, 539, 468, 611, 581, 600, 594, 703, 711, 849, 981, 944, 865, 850, 883, 881, 1221, 1137, 1224, 1146, 1282, 1248, 1179, 1424, 1359, 1528, 1484, 1571, 1727, 1732]
Topics:
[(2, 0.99992562868881996)]

JamesM.Bower
Docs: [17, 48, 58, 131, 101, 126, 127, 281, 208, 225]
Topics:
[(5, 0.99980243307117478)]


We can construct the `doc2author` dictionary ourselves.

In [20]:
from gensim.models import atmodel
doc2author = atmodel.construct_doc2author(author2doc=author2doc, corpus=corpus)

We can also compute the per-word bound, which is a measure of the model's predictive performance (you could also say that it is the reconstruction error).

In [21]:
# Compute the per-word bound.
# Number of words in corpus.
corpus_words = sum(cnt for document in corpus for _, cnt in document)

# Compute bound and divide by number of words.
perwordbound = model.bound(corpus, author2doc=author2doc, doc2author=doc2author) / corpus_words
print(perwordbound)

-7.75230922299


We can evaluate the quality of the topics by computing the topic coherence, as in the LDA class. Use this to e.g. find out which of the topics are poor quality, or as a metric for model selection.

In [22]:
%time top_topics = model.top_topics(corpus)

CPU times: user 15.8 s, sys: 12 ms, total: 15.8 s
Wall time: 15.8 s


#### Plotting the authors

Now we're going to produce the kind of pacific archipelago looking plot below. The goal of this plot is to give you a way to explore the author-topic representation in an intuitive manner.

We take all the author-topic distributions (stored in `model.state.gamma`) and embed them in a 2D space. To do this, we reduce the dimensionality of this data using t-SNE. 

t-SNE is a method that attempts to reduce the dimensionality of a dataset, while maintaining the distances between the points. That means that if two authors are close together in the plot below, then their topic distributions are similar.

In the cell below, we transform the author-topic representation into the t-SNE space. You can increase the `smallest_author` value if you do not want to view all the authors with few documents.

In [26]:
%%time
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
smallest_author = 0  # Ignore authors with documents less than this.
authors = [model.author2id[a] for a in model.author2id.keys() if len(author2doc[a]) > smallest_author]
_ = tsne.fit_transform(model.state.gamma[authors, :])  # Result stored in tsne.embedding_

We are now ready to make the plot.

If you are unable to view or interact with the plot below, it is available [here]() (**TODO:** make a page for the plot, and include the link), or view the entire notebook [here]() (**TODO:** make nvbiewer page for the notebook or something).

Note that if you run this notebook yourself, you will see a different graph. The random initialization of the model will be different, and the result will thus be different to some degree. You may find an entirely different representation of the data, or it may show the same interpretation slightly differently.

In [27]:
# Tell Bokeh to display plots inside the notebook.
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [28]:
from bokeh.models import HoverTool
from bokeh.plotting import figure, show, ColumnDataSource

x = tsne.embedding_[:, 0]
y = tsne.embedding_[:, 1]
author_names = [model.id2author[a] for a in authors]

# Radius of each point corresponds to the number of documents attributed to that author.
scale = 0.1
author_sizes = [len(author2doc[a]) for a in author_names]
radii = [size * scale for size in author_sizes]

source = ColumnDataSource(
        data=dict(
            x=x,
            y=y,
            author_names=author_names,
            author_sizes=author_sizes,
            radii=radii,
        )
    )

# Add author names and sizes to mouse-over info.
hover = HoverTool(
        tooltips=[
        ("author", "@author_names"),
        ("size", "@author_sizes"),
        ]
    )

p = figure(tools=[hover, 'crosshair,pan,wheel_zoom,box_zoom,reset,save,lasso_select'])
p.scatter('x', 'y', radius='radii', source=source, fill_alpha=0.6, line_color=None)
show(p)

The circles in the plot above are individual authors, and their sizes represent the number of documents attributed to the corresponding author. Hovering your mouse over the circles will tell you the name of the authors and their sizes.

**FIXME:** give interpretation of clusters in plot.

Clusters of large authors tend to reflect some overlap in interest. 

### Advanced topics

This section describes some of the more advanced use cases of the model.

**FIXME:** if there aren't other "advanced topics", remove that headline.

#### Serialized corpora

The `AuthorTopicModel` class accepts serialized corpora, that is, corpora that are stored on the hard-drive rather than in memory. This is usually done when the corpus is too big to fit in memory. There are, however, some caveats to this functionality, which we will discuss here. As these caveats make this functionality less than ideal, it may be improved in the future.

In the following, an explanation, followed by an example and a summarization will be given.

If the corpus is serialized, the user must specify `serialized=True`. Any input corpus can then be any type of iterable or generator.

The model will then take the input corpus and serialize it in the `MmCorpus` format, which is [supported in Gensim](https://radimrehurek.com/gensim/corpora/mmcorpus.html).

The user must specify the path where the model should serialize all input documents, for example `serialization_path='/tmp/model_serializer.mm'`. To avoid accidentally overwriting some important data, the model will raise an error if there already exists a file at `serialization_path`; in this case, either choose another path, or delete the old file.

When you want to train on new data, and call `model.update(corpus, author2doc)`, all the old data and the new data have to be re-serialized. This can of course be quite computationally demanding, so it is recommended that you do this *only* when necessary; that is, wait until you have as much new data as possible to update, rather than updating the model for every new document.

In [ ]:
%time model_ser = AuthorTopicModel(corpus=corpus, num_topics=10, id2word=dictionary.id2token, \
                               author2doc=author2doc, random_state=1, serialized=True, \
                               serialization_path='/tmp/model_serialization.mm')

In [ ]:
# Delete the file, once you're done using it.
import os
os.remove('/tmp/model_serialization.mm')

In summary, when using serialized corpora:
* Set `serialized=True`.
* Set `serialization_path` to a path that doesn't already contain a file.
* Wait until you have lots of data before you call `model.update(corpus, author2doc)`.
* When done, delete the file at `serialization_path` if it's not needed anymore.